# GeNet Experiments Code

In [ ]:
! pip install tiktoken
! pip install cohere
! pip install docx
! pip install exceptions
! pip install python-docx

In [ ]:
%%capture
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

In [ ]:
import os
os.kill(os.getpid(), 9)

restart runtime after the cell above (in order to solve openAI API issues (as of date Dec 2024) )

In [ ]:
!git clone https://github.com/networkcopilot/GeNet-Framework.git

In [ ]:
from openai import OpenAI
from google.colab import userdata
api_key = userdata.get('api_key')
client = OpenAI(api_key = api_key)

In [ ]:
import os
os.makedirs("/content/GeNet_Experiment_Results", exist_ok=True)

In [ ]:
run_number = 1 # <--this should change according to your itteration number

## Topology Understanding using GPT4Turbo

**The prompt for this task:**
>  As a network topology analyst, provide a detailed description of the components and edges in the given diagram.
Ensure the description includes: (1) the specific icon of each component (if icons are present), such as router, pc, firewall, etc. (2) a detailed list of all edges with both edge interfaces described if edge labels are present.
Avoid adding any extra comments or interpretations about the diagram.



### GPT4V request code
**NOTICE!** The given image **MUST** be in jpeg format for it to work correctly!

as you can see the url of the given image in this code is in a jpeg format - this does effect the correctness of the results.

In [ ]:
import base64
import requests
from google.colab import userdata
api_key = userdata.get('api_key')#need to add the key to the 'secrets section'
# OpenAI API Key
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')


# Getting the base64 string

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

def create_image_content(_image_path):
    return {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{encode_image(_image_path)}",
            "detail": "high"
          }
        }
def make_request(model_name, input_text, temperature, image_content, image_path=None, max_tokens=300):
  content = [
        {
          "type": "text",
          "text": input_text
        }]

  content.append(image_content)
  payload = {
    "model": model_name,
    "messages": [
      {
        "role": "user",
        "content": content

      }
    ],
    "max_tokens": max_tokens,
    "temperature": temperature,
    "top_p": 0.1
  }

  return requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

In [ ]:
from inspect import EndOfBlock
import os
import pandas as pd
import time


topology_understanding_time_calc_file_address = f"/content/topology_understanding_time_Run{run_number}.csv"
if os.path.isfile(topology_understanding_time_calc_file_address):
  topology_understanding_time_df = pd.read_csv(topology_understanding_time_calc_file_address)
else:
  topology_understanding_time_df = pd.DataFrame(columns=["Scenario","Visualization","Diagram_Type","Run","Time"])

model_name2 = "gpt-4-turbo"
temp = 1.0
diagram_types = ["Messy_Layout", "No_Labels_On_Edges", "Normal"]
visualization_formats =["GNS3","Paper_Sketches","PowerPoint"]

#--------------------------------------------------------------------------------

genet_dataset_dir = "/content/GeNet-Framework/Dataset"
topology_understanding_output_dir = "/content/topology_understanding_output"
os.makedirs(topology_understanding_output_dir, exist_ok=True)
scenarios = {
    "Configuration Scenarios": ["Role_Based_CLI_Access","Time_Based_Access_List","Transparent_IOS_Firewall", "Basic_Zone_Based_Firewall", "IP_Traffic_Export"],
    "Topology Scenarios": ["Adding_Communication_Servers", "Adding_DMZ", "Adding_DRA", "Adding_Local_PCs", "Internet_Connectivity"]
            }
input_text ="""As a network topology analyst, provide a detailed description of the components and edges in the given diagram.
Ensure the description includes: (1) the specific icon of each component (if icons are present), such as router, pc, firewall, etc. (2) a detailed list of all edges with both edge interfaces described if edge labels are present.
Avoid adding any extra comments or interpretations about the diagram."""

for scenario_type, scenario_list in scenarios.items():
  topology_image_variance_dir = os.payh.join(genet_dataset_dir, scenario_type, "Topology_image_variance")
  result_scenario_type_dir = os.path.join(topology_understanding_output_dir, scenario_type)
  os.makedirs(result_scenario_type_dir, exist_ok=True)

  for visualization in visualization_formats:
    visualization_format_dir = os.path.join(topology_image_variance_dir, visualization)
    result_visualization_dir = os.path.join(result_scenario_type_dir, visualization)
    os.makedirs(result_visualization_dir, exist_ok=True)
  for diagram_type in diagram_types:
    diagram_type_dir_path = os.path.join(visualization_format_dir, diagram_type)
    result_diagram_type_dir = os.path.join(result_visualization_dir, diagram_type)
    os.makedirs(result_diagram_type_dir, exist_ok=True)
    for scenario in scenario_list:

      scenario_dir = os.path.join(diagram_type_dir_path, scenario)
      result_scenario_dir = os.path.join(result_diagram_type_dir, scenario)
      os.makedirs(result_scenario_dir, exist_ok=True)
      graph_image_path = f"{scenario_dir}/{scenario}_{visualization}_{diagram_type}.jpg"
      img_content = create_image_content(graph_image_path)
      ans_file_path = os.path.join(result_scenario_dir,f"Topology(Run{run_number}).txt")
      if os.path.isfile(ans_file_path):
        continue
      print(f"\n--------------VISUALIZATION: {visualization}, DIAGRAM TYPE: {diagram_type}, SCENARIO: {scenario}, RUN:{run_number} ----------------------------------\n")
      start_time = time.time()
      gpt_answer = make_request(model_name=model_name2, input_text=input_text, temperature=temp, image_content=img_content, image_path=graph_image_path, max_tokens=2048).json()
      answer = gpt_answer["choices"][0]
      end_time = time.time()
      time_taken = end_time-start_time
      new_row = {"Scenario":scenario, "Visualization":visualization, "Diagram_Type":diagram_type, "Run":run_number, "Time":time_taken}
      topology_understanding_time_df.loc[len(topology_understanding_time_df)] = new_row
      ans_file_path = os.path.join(result_scenario_dir,f"Topology(Run{run_number}).txt")
      text_file = open(ans_file_path, "w")
      text_file.write(answer['message']['content'])
      text_file.close()
      print(answer['message']['content'])
topology_understanding_time_df.to_csv(topology_understanding_time_calc_file_address, index=False)


## Intent Implementation Module

**The instructions for the Intent Implementation Assistant:**
> You are a network architecture and configuration expert.
You are given 2 files and an intent by a user, one of the given files contains a textual representation of the network topology, and the other contains the configurations of the network devices.
Given the files and an intent, your mission is to modify those files so that the network adheres to the user intent.
If a change to the topology is required, please update the file of the textual representation of the topology accordingly. If the change is an addition of a network component, please configure it and add the configuration to the configurations file.
After modifying a file, you provide the user a way to access the updated file to download it.
In your answer to the user, you should provide brief explanations of the actions, changes, and updates you perform.
If at any point in your messaging thread you do not have access to all the given files, state this and stop answering.

In [ ]:
intent_implementation_assistant_instructions = """You are a network architecture and configuration expert.
You are given 2 files and an intent by a user, one of the given files contains a textual representation of the network topology, and the other contains the configurations of the network devices.
Given the files and an intent, your mission is to modify those files so that the network adheres to the user intent.
If a change to the topology is required, please update the file of the textual representation of the topology accordingly. If the change is an addition of a network component, please configure it and add the configuration to the configurations file.
After modifying a file, you provide the user a way to access the updated file to download it.
In your answer to the user, you should provide brief explanations of the actions, changes, and updates you perform.
If at any point in your messaging thread you do not have access to all the given files, state this and stop answering."""

In [ ]:
my_assistant = client.beta.assistants.create(
    instructions= intent_implementation_assistant_instructions,
    name="Network Architecture Expert",
    tools=[{"type": "code_interpreter"}, {"type": "file_search"}],
    model="gpt-4-turbo",
)
assist_num = my_assistant.id
print(my_assistant)

**The prompt for this task:**
> Hello network architecture expert, Here you were given two text files: a full configuration file named “{configuration_file_name}” and a textual representation of the topology named “{textual_topology_name}” Here is my intent: {intent}. Please ensure that you read both of the provided files entirely and make the necessary modifications accordingly, apply the modifications without waiting for confirmation for your actions.
        

In [ ]:
def create_thread(number, intent, textual_pr_file, config_file):

  configuration_file_name = "Total_Configs.txt"
  textual_topology_name = f'Topology(Run{number}).txt'


  message_thread = client.beta.threads.create(
    messages=[
      {
        "role": "user",
        "content": f'Hello network architecture expert, Here you were given two text files: a full configuration file named “{configuration_file_name}” and a textual representation of the topology named “{textual_topology_name}” Here is my intent: {intent}. Please ensure that you read both of the provided files entirely and make the necessary modifications accordingly, apply the modifications without waiting for confirmation for your actions.',
        #----------------- ADDITIONAL LINES FOR NEW API CODE-----------------------------
        "attachments": [
        {"file_id": config_file.id, "tools": [{"type": "file_search"},{"type": "code_interpreter"}]},
        {"file_id": textual_pr_file.id, "tools": [{"type": "file_search"},{"type": "code_interpreter"}]}
        ],
        #-----------------------------------------------------------------------------------
      },
    ]
  )
  return message_thread

In [ ]:
import os
import shutil
import os.path
import time
import pandas as pd


intent_implementation_time_calc_file_address = f"/content/Intent_implementation_time_run{run_number}.csv"
if os.path.isfile(intent_implementation_time_calc_file_address):
  intent_implementation_time_df = pd.read_csv(intent_implementation_time_calc_file_address)
else:
  intent_implementation_time_df = pd.DataFrame(columns=["Scenario","Visualization","Diagram_Type" "Run","Time"])

scenarios = {
    "Configuration Scenarios": ["Role_Based_CLI_Access","Time_Based_Access_List","Transparent_IOS_Firewall", "Basic_Zone_Based_Firewall", "IP_Traffic_Export"],
    "Topology Scenarios": ["Adding_Communication_Servers", "Adding_DMZ", "Adding_DRA", "Adding_Local_PCs", "Internet_Connectivity"]
            }
topology_understanding_output_dir = "/content/topology_understanding_output"
intent_implementation_output_dir = "/content/intent_implementation_results"
genet_dataset_dir = "/content/GeNet-Framework/Dataset"

def intent_implementation_execution(row):

  if row["Intent_Implementation"] == "V":
    return row
  visualization = row["Visualization"]
  diagram_type = row["Diagram_Type"]
  scenario = row["Scenario"]
  scenario_type = row["Scenario_Type"]
  visualization_dir = os.path.join(intent_implementation_output_dir, scenario_type, visualization)
  diagram_type_dir = os.path.join(visualization_dir, diagram_type)
  intent_path = os.path.join(genet_dataset_dir, scenario_type, "Initial files", scenario, "intent.txt")
  config_path = os.path.join(genet_dataset_dir, scenario_type, "Initial files", scenario, "Total_Configs.txt")
  intent = open(intent_path, 'r').read()
  scenario_folder = os.path.join(diagram_type_dir, scenario)
  os.makedirs(scenario_folder, exist_ok=True)
  print(f"\n---------------------[VISUALIZATION: {visualization}, DIAGRAM TYPE: {diagram_type}, SCENARIO: {scenario}, RUN:{run_number}]--------------\n")

  # upload the configuration file to openAI
  config_file = client.files.create(
    file=open(config_path, "rb"),
    purpose='assistants'
    )

  run_folder = os.path.join(scenario_folder, f"Run{run_number}")
  os.makedirs(run_folder, exist_ok=True)
  # upload the topology understanding result file to openAI
  topology_file = f"{topology_understanding_output_dir}/{visualization}/{diagram_type}/{scenario}/Topology(Run{run_number}).txt"
  textual_pr_file = client.files.create(
    file=open(topology_file, "rb"),
    purpose='assistants'
  )
  # create a new thread in the assistant
  message_thread = create_thread(run_number, intent, textual_pr_file, config_file)
  thread_id = message_thread.id

  # run the thread
  start_time = time.time()
  run = client.beta.threads.runs.create_and_poll(
    temperature = 1.0,
    thread_id = thread_id,
    assistant_id = assist_num
  )

  # check if the run is completed
  if run.status == 'completed':
    end_time = time.time()
    time_taken = end_time-start_time

    messages = client.beta.threads.messages.list(
    thread_id=thread_id
    )

    #getting the run results attachments:
    attachments =[]
    for message in messages.data:
      if message.role == "assistant":
        messg_id = message.id
        value = message.content[0].text.value
        print(value)

        if message.attachments:
            for attachment in message.attachments:
                file = client.files.retrieve(attachment.file_id)
                if file.purpose == "assistants_output":
                    content = client.files.content(file.id)
                    file_name = file.filename
                    # removing extra edditions from the generated file
                    new_file_name = file_name.rsplit('/')[-1]
                    new_file_name = new_file_name
                    with open(f"{run_folder}/{new_file_name}", "wb") as f:
                        f.write(content.read())
                    attachments.append(new_file_name)
    print("\n\n")
    if attachments:
      for name in attachments:
        print(f"downloaded file named {name}\n")
      row["Intent_Implementation"] = "V"
    else:
      print("ERROR: Attachments not found \n")
    new_row = {"Scenario_Type": scenario_type ,"Scenario":scenario, "Visualization":visualization, "Diagram_Type":diagram_type,"Run":run_number, "Time":time_taken}

    intent_implementation_time_df.loc[len(intent_implementation_time_df)] = new_row

  # in case the run crashed for some reason:
  else:
    print(f"Run {run_number} failed with status {run.status}")
    new_row = {"Scenario_Type": scenario_type ,"Scenario":scenario, "Visualization":visualization, "Diagram_Type":diagram_type, "Run":run_number, "Time":0}
    intent_implementation_time_df.loc[len(intent_implementation_time_df)] = new_row
    row["Intent_Implementation"] = "X"
  print("\n---------------------------------------------------------\n")

  time.sleep(5)

  intent_implementation_time_df.to_csv(intent_implementation_time_calc_file_address, index=False)
  return row


if not os.path.isfile(f"/content/Run{run_number}_tracking.csv"):
  df = pd.read_csv("/content/Dataset/specs/GeNet_result_template.csv")
else:
  df = pd.read_csv(f"/content/Run{run_number}_tracking.csv")

df_dict = df.to_dict('index')
for index, row in df_dict.items():
  row = intent_implementation_execution(row)
df = pd.DataFrame.from_dict(df_dict, orient='index')
csv_file_path = f"/content/Intent_implementation_time_run{run_number}.csv" # Specify the desired file path
df.to_csv(csv_file_path, index=False)

## Topology Understanding text to json and evaluation

In [ ]:
from pathlib import Path
import json

def parse_topology_from_response(natural_language_response):
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        response_format={ "type": "json_object" },
        messages=[
            {
                "role": "system",
                "content": """
                You will recieve a textual description of a network topology. Convert it to a JSON format as follows:
                {
                  "nodes": [
                    {"label":<NODE1>,
                     "icon": <the respective icon, one of [router, firewall, ethernet_switch, pc, cloud, ids]>
                     //Notice! If needed, you should change the icon name to one of the options above, for example "Desktop computer symbol" should change to "pc".
                    },
                    ...
                    ] // This is a list of all the device names mentioned in the topology(routers, firewalls, switches, pc, etc)
                     "links": [[<source_device>, <destination_device>, <source_network_interface>, <destination_network_interface>],...] // This is a list of all the connections/edges mentioned in the topology, each connection represented as an array
               }
               Notice! if no interfaces are mentioned in the textual description please leave an empty string ("") in the <source_network_interface>, <destination_network_interface> fields
                 """
            },
            {
                "role": "user",
                "content": f"{natural_language_response}"

            }
        ],
        temperature = 0.2, #setting the temperature to a higher than zero value so the LLM can handle more complex textual description
        top_p = 0.1
    )

    return json.loads(response.choices[0].message.content)


In [ ]:
os.makedirs("/content/topology_understanding_result_parsed_to_json", exist_ok=True)
parsing_dir_path = "/content/topology_understanding_result_parsed_to_json"



def topology_understanding_eval_function(row):
  if row["Topology_Understanding_Eval"] == "V":
    return row
  visualization = row["Visualization"]
  diagram_type = row["Diagram_Type"]
  scenario = row["Scenario"]
  scenario_type = row["Scenario_Type"]
  visualization_diagram_dir = os.path.join(topology_understanding_output_dir, visualization)
  diagram_type_dir = os.path.join(visualization_diagram_dir, diagram_type)
  scenario_dir = os.path.join(diagram_type_dir, scenario)
  topology_text_file = f"{topology_understanding_output_dir}/{scenario_type}/{visualization}/{diagram_type}/{scenario}/Topology(Run{run_number}).txt"
  file_path = Path(topology_text_file)
  output_file_path = f"{parsing_dir_path}/{scenario_type}/{visualization}/{diagram_type}/{scenario}/Parsed_Topology(Run{run_number}).json"
  if os.path.isfile(output_file_path):
    print(f"{visualization}_{diagram_type}_{scenario} : {run_number} done. \n")
    row["Topology_Understanding_Eval"] = "V"
    return row
  file_content = file_path.read_text()
  parsed_topology = parse_topology_from_response(file_content)
  with open(output_file_path, 'w', encoding='utf-8') as f:
      json.dump(parsed_topology, f, ensure_ascii=False, indent=4)
  print(f"{visualization}_{diagram_type}_{scenario} : {run_number} done. \n")


df = pd.read_csv(f"/content/Run{run_number}_tracking.csv")
df_dict = df.to_dict('index')
for index, row in df_dict.items():
  row = topology_understanding_eval_function(row)
df = pd.DataFrame.from_dict(df_dict, orient='index')
csv_file_path = f"/content/Run{run_number}_tracking.csv" # Specify the desired file path
df.to_csv(csv_file_path, index=False)

### Running comparison between Topology Understanding results and Ground truth

In [ ]:
from itertools import permutations, combinations

def get_equal_pairs(arr1, arr2, attr="label"):
    equals = []

    # For each product set {i,j}|i!=j , add to array if strings are identical
    for i in range(0,len(arr1)):
        for j in range(0, len(arr2)):
            if arr1[i][attr].lower() == arr2[j][attr].lower():
              equals = equals + [(arr1[i], arr2[j])]

    return equals

def get_sim_NPL(nodes1, nodes2):
    sim_NPL = get_equal_pairs(arr1 = nodes1, arr2 = nodes2, attr="label")
    return sim_NPL

#def get_sim_NPI(nodes1, nodes2, sim_NPL):
def get_sim_NPI(sim_NPL):
    sim_NPI = []
    for pair in sim_NPL:
        node1 = pair[0]
        node2 = pair[1]
        if  node1["icon"].lower() == node2["icon"].lower():
          sim_NPI = sim_NPI + [pair]

    return sim_NPI

def are_connected(node1, node2, links):
    undirected_links = [set([link[0], link[1]]) for link in links]
    query_link = set([node1["label"], node2["label"]])
    if query_link in undirected_links:
        return True

    return False

def get_edge(node1, node2, links):
    node1_label = node1["label"]
    node2_label = node2["label"]
    possible_links = [link for link in links if set([node1_label, node2_label]) == set([link[0], link[1]])]
    link = possible_links[0]

    return link

def get_sim_E(sim_NPL, links1, links2):
    sim_E = 0
    sim_NPL_pairs = combinations(sim_NPL, 2)
    for sim_NPL_pair in sim_NPL_pairs:
        sim_pair1 = sim_NPL_pair[0]
        sim_pair2 = sim_NPL_pair[1]
        if are_connected(sim_pair1[0], sim_pair2[0], links=links1) and are_connected(sim_pair1[1], sim_pair2[1], links=links2):
            sim_E = sim_E + 1

    return sim_E

def get_sim_EPlabel(sim_NPL, links1, links2):
    sim_EPlabel = 0
    sim_NPL_pairs = combinations(sim_NPL, 2)
    for sim_NPL_pair in sim_NPL_pairs:
        sim_pair1 = sim_NPL_pair[0]
        sim_pair2 = sim_NPL_pair[1]
        connected_by_edge = are_connected(sim_pair1[0], sim_pair2[0], links=links1) and are_connected(sim_pair1[1], sim_pair2[1], links=links2)
        if connected_by_edge:
            edge1 = get_edge(sim_pair1[0], sim_pair2[0], links1)
            edge2 = get_edge(sim_pair1[1], sim_pair2[1], links2)
            edge1_interfaces = sorted([str(edge1[2]),str(edge1[3])])
            edge2_interfaces = sorted([str(edge2[2]), str(edge2[3])])
            if edge1_interfaces[0].lower() == edge2_interfaces[0].lower():
                sim_EPlabel = sim_EPlabel + 1
            if edge1_interfaces[1].lower() == edge2_interfaces[1].lower():
                sim_EPlabel = sim_EPlabel + 1

    return sim_EPlabel

def total_nodes(topology):
    return len(topology["nodes"])

def total_edges(topology):
    return len(topology["links"])


# strip labels spaces
def strip_node_names(nodes):
  for node in nodes:
    node["label"] = node["label"].replace(" ","")
  return nodes

#strip edges spaces
def strip_edge_names(links):
  for link in links:
    link[0] = link[0].replace(" ","")
    link[1] = link[1].replace(" ","")
  return links


def get_comparison_metrics(topology1, topology2):
    N_coe = 0.3
    NPL_coe =0.2
    NPI_coe = 0.05
    E_coe = 0.35
    EPlabel_coe = 0.1

    # remove spaces in names
    topology1["nodes"] = strip_node_names(topology1["nodes"])
    topology2["nodes"] = strip_node_names(topology2["nodes"])
    topology1["links"] = strip_edge_names(topology1["links"])
    topology2["links"] = strip_edge_names(topology2["links"])

    N1, N2 = total_nodes(topology1), total_nodes(topology2)
    if N1 == 0 or N2 == 0:
        raise Exception("N1 or N2 is 0, Cannot divide by 0")

    E1, E2 = total_edges(topology1), total_edges(topology2)
    if E1 == 0 or E2 == 0:
        raise Exception("E1 or E2 is 0, Cannot divide by 0")

    sim_NPL = get_sim_NPL(topology1["nodes"], topology2["nodes"])
    NPL = len(sim_NPL) / max(N1, N2)
    sim_E = get_sim_E(sim_NPL, topology1["links"], topology2["links"])
    sim_NPI = get_sim_NPI(sim_NPL=sim_NPL)
    NPI = len(sim_NPI) / max(N1, N2)
    sim_EPlabel = get_sim_EPlabel(sim_NPL, topology1["links"], topology2["links"])

    Metric_results_value = N_coe*min(N1,N2)/max(N1,N2) + NPL_coe*NPL + NPI_coe*NPI + E_coe*sim_E/max(E1,E2) + EPlabel_coe*sim_EPlabel/(2*max(E1,E2))
    return {'N1':N1, 'N2':N2, 'E1':E1, 'E2':E2, 'NPL':NPL, 'NPI':NPI, 'sim_E':sim_E, 'sim_EPlabel':sim_EPlabel, 'Metric_score':Metric_results_value}




In [ ]:
import pprint
import json
import time
import pandas as pd

parsing_dir_path = "/content/topology_understanding_result_parsed_to_json"
diagram_types = ["Messy_Layout", "No_Labels_On_Edges", "Normal"]
visualization_formats =["GNS3",  "Paper_Sketches", "PowerPoint"] #"Packet_Tracer", "Visio"
scenarios = {
    "Configuration Scenarios": ["Role_Based_CLI_Access","Time_Based_Access_List","Transparent_IOS_Firewall", "Basic_Zone_Based_Firewall", "IP_Traffic_Export"],
    "Topology Scenarios": ["Adding_Communication_Servers", "Adding_DMZ", "Adding_DRA", "Adding_Local_PCs", "Internet_Connectivity"]
            }
temp = 1.0
results_list = []





for scenario_type, scenario_list in scenarios.items():
  for visualization in visualization_formats:
    for diagram_type in diagram_types:
      for scenario in scenario_list:

        gns3_file_path = f"/content/GeNet-Framework/Dataset/{scenario_type}/gns3_topology_parsing/{scenario}.json"
        with open(gns3_file_path, 'r') as f:
          benchmark_topology = json.load(f)
        topology_understanding_scenario_directory = f"{parsing_dir_path}/{scenario_type}/{visualization}/{diagram_type}/{scenario}"

        print(f"Running {visualization} {diagram_type} {scenario} {run_number} \n")
        with open(f"{topology_understanding_scenario_directory}/Parsed_Topology(Run{run_number}).json", 'r') as f:
          openai_topology = json.load(f)
        result = get_comparison_metrics(benchmark_topology, openai_topology)
        result["Scenario_Type"] = scenario_type
        result["Scenario"] = scenario
        result["Visualization"] = visualization
        result["Diagram_Type"] = diagram_type
        result["Run"] = run_number

        results_list.append(result)
        pprint.pprint(result)
        print("--------------------------\n")


with open(f"/{parsing_dir_path}/Topology_Understanding_Run{run_number}_evaluation_results.json", "w") as f:
  json.dump(results_list, f, ensure_ascii=False, indent=4)

df = pd.DataFrame(results_list)
df.to_csv(f"/{parsing_dir_path}/Topology_Understanding_Run{run_number}_evaluation_results.csv", index=False)

## Intent Implementation Evaluation

### Evaluation:


**Instructions for LLM-Based Evaluation Assistant:**
>You are a helpful networking intent implementation evaluator assistant. Your role as an evaluator is to assess the accuracy of network intent implementation by grading it. You will review the provided files:  
1. Initial Network Components Configuration – The starting configuration of network devices.  
2. Initial Topology Description – The original structure of the network.  
3. Textual Intent – A detailed description of the desired changes or outcomes for the network's configuration.  
4. Updated Network Components Configuration files or topology files – Several files which show the changes made to the topology or to the configuration based on the intent. The number of those files can be one or more.

>Evaluation Process:
1. Understand the Intent: Carefully analyze the textual intent to fully grasp the desired changes or outcomes.  
2. Analyze Initial Files: Examine the initial network configuration and topology to understand the starting state and context.  
3. Evaluate Updated Files: Compare the updated configurations against the intent to determine if the desired changes have been accurately implemented.  

>Scoring and Grading:  
- Scoring Keys: You will be provided with specific scoring keys by the user to guide your evaluation. Each of those keys specifies a condition for noncompliance with the intent implementation, and the point deductions for this specific noncompliance.
	- Points should only be deducted if the updated file complies with the key’s condition (i.e., if the issue described in the key is present in the implementation).
	- Partial deductions are allowed for partial compliance or minor issues related to a key.  
- Grading:  
	- Start with a grade of 100.
	- If the implementation meets the noncompliance criteria described in a key, subtract the corresponding points from the grade.
	- Ensure deductions align strictly with the provided scoring keys and do not penalize issues outside their scope.  

>Response Guidelines:  
- Provide the final grade as a numeric value (0–100).  
- Adhere strictly to these instructions and the provided scoring keys during the evaluation process.





In [ ]:
implementation_evaluator_assistant_instructions = """You are a helpful networking intent implementation evaluator assistant. Your role as an evaluator is to assess the accuracy of network intent implementation by grading it. You will review the provided files:
1. Initial Network Components Configuration – The starting configuration of network devices.
2. Initial Topology Description – The original structure of the network.
3. Textual Intent – A detailed description of the desired changes or outcomes for the network's configuration.
4. Updated Network Components Configuration files or topology files – Several files which show the changes made to the topology or to the configuration based on the intent. The number of those files can be one or more.

Evaluation Process:
1. Understand the Intent: Carefully analyze the textual intent to fully grasp the desired changes or outcomes.
2. Analyze Initial Files: Examine the initial network configuration and topology to understand the starting state and context.
3. Evaluate Updated Files: Compare the updated configurations against the intent to determine if the desired changes have been accurately implemented.

Scoring and Grading:
- Scoring Keys: You will be provided with specific scoring keys by the user to guide your evaluation. Each of those keys specifies a condition for noncompliance with the intent implementation, and the point deductions for this specific noncompliance.
	- Points should only be deducted if the updated file complies with the key’s condition (i.e., if the issue described in the key is present in the implementation).
	- Partial deductions are allowed for partial compliance or minor issues related to a key.
- Grading:
	- Start with a grade of 100.
	- If the implementation meets the noncompliance criteria described in a key, subtract the corresponding points from the grade.
	- Ensure deductions align strictly with the provided scoring keys and do not penalize issues outside their scope.

Response Guidelines:
- Provide the final grade as a numeric value (0–100).
- Adhere strictly to these instructions and the provided scoring keys during the evaluation process."""

In [ ]:
my_assistant = client.beta.assistants.create(
    instructions=implementation_evaluator_assistant_instructions,
    name="Networking Intent Implementation Evaluator",
    tools=[{"type": "file_search"}],
    model="gpt-4o",
    temperature = 0.1
)
assist_num = my_assistant.id
print(my_assistant)

**The prompt for this task:**
> Hello Networking Intent Implementation Evaluator, I added here {file_number} files: '{input_configuration_file_name}' which is the initial network component configuration file, '{input_textual_topology_name}' which is the initial topology description, an intent file named 'intent.txt', and {len(files)} updated files which shows the changes after intent implementation named {files_names}.
Here are the scoring keys for your evaluation:
{scoring_keys}
Given the these files, and the scoring keys, assign a grade to the intent implementation according to your instructions, and briefly explain your scoring for each key.,


In [ ]:
def create_thread_topology_scenario(intent, input_config, input_topology, input_topology_name, updated_files_path, scoring_keys_address):

  input_configuration_file_name = 'Total_Configs.txt'
  input_textual_topology_name = f'{input_topology_name}.json'

  files = [f for f in os.listdir(updated_files_path) if os.path.isfile(os.path.join(updated_files_path, f))]
  updated_files = []
  for file in files:
    file_path = os.path.join(updated_files_path, file)
    assistant_file = client.files.create(
      file=open(file_path, "rb"),
      purpose='assistants'
      )
    updated_files.append((file, assistant_file.id))

  input_config_file = client.files.create(
    file=open(input_config, "rb"),
    purpose='assistants'
    )
  input_topology_file = client.files.create(
    file=open(input_topology, "rb"),
    purpose='assistants'
    )
  intent_file = client.files.create(
    file=open(intent, "rb"),
    purpose='assistants'
    )
  with open( scoring_keys_address, 'r') as file:
    scoring_keys = file.read()

  attachments = [
        {"file_id": input_config_file.id, "tools": [{"type": "file_search"}]},
        {"file_id": input_topology_file.id, "tools": [{"type": "file_search"}]},
        {"file_id": intent_file.id, "tools": [{"type": "file_search"}]}
        ]
  for file in updated_files:
    attachments.append({"file_id": file[1], "tools": [{"type": "file_search"}]})

  file_number = 3+len(files)

  if len(files) ==1:
    updated_file_message = f"and an updated file which shows the changes after intent implementation named {updated_files[0][0]}"
  else:
    files_names = ""
    for i in range(len(files)-1):
      files_names = files_names+f"{updated_files[i][0]}, "
    files_names = files_names+f" and {updated_files[-1][0]}"
    updated_file_message = f"and {len(files)} updated files which shows the changes after intent implementation named {files_names}"


  message_thread = client.beta.threads.create(
    messages=[
      {
        "role": "user",
        "content": f"""Hello Networking Intent Implementation Evaluator, I added here {file_number} files: '{input_configuration_file_name}' which is the initial network component configuration file, '{input_textual_topology_name}' which is the initial topology description, an intent file named 'intent.txt', {updated_file_message}.
Here are the scoring keys for your evaluation:
{scoring_keys}
Given the these files, and the scoring keys, assign a grade to the intent implementation according to your instructions, and briefly explain your scoring for each key.""",
        #----------------- ADDITIONAL LINES FOR NEW API CODE-----------------------------
        "attachments": attachments,
        #-----------------------------------------------------------------------------------
      },
    ]
  )
  return message_thread

In [ ]:
import os
import pandas as pd


if not os.path.isfile(f"/content/Run{run_number}_tracking.csv"):
  itterations_file = pd.read_csv("/content/Dataset/specs/GeNet_result_template.csv")
else:
  itterations_file = pd.read_csv(f"/content/Run{run_number}_tracking.csv")


genet_dataset_dir = "/content/GeNet-Framework/Dataset"
results_folder = "/content/Verifier_LLM_Evaluator_with_descriptions"
intent_implementation_output_dir = "/content/intent_implementation_results"
os.makedirs(results_folder, exist_ok=True)



for index, row in itterations_file.iterrows():
  if row["Intent_Implementation"] != "V" or row["Intent_Implementation_Eval"]=="V": # failed run or not yet graded
    continue

  scenario_type = row["Scenario_Type"]
  visualization = row["Visualization"]
  diagram_type = row["Diagram_Type"]
  scenario = row["Scenario"]

  scoring_keys_directory = os.path.join(genet_dataset_dir, scenario_type, "Scoring_Keys")
  output_dir_address = os.path.join(results_folder, scenario_type, visualization, diagram_type)
  os.makedirs(output_dir_address, exist_ok=True)
  output_file_address = os.path.join(output_dir_address, f"Run{run_number}_results.txt") # <--- one result file per run
  output_file = open(output_file_address, "a")

  intent_file_address = os.path.join(genet_dataset_dir, scenario_type, "Initial files", scenario, "intent.txt")
  input_config_file_address = os.path.join(genet_dataset_dir, scenario_type, "Initial files", scenario, "Total_Configs.txt")
  updated_specs_scenario_directory = os.path.join(intent_implementation_output_dir, scenario_type, visualization, diagram_type, scenario)
  scoring_keys_address = os.path.join(scoring_keys_directory, f"{scenario}.txt")

  updated_spec_run_directory = os.path.join(updated_specs_scenario_directory,f"Run{run_number}")
  input_topology_file_address = os.path.join(genet_dataset_dir, scenario_type, "gns3_topology_parsing", f"{scenario}.json")


  message_thread = create_thread_topology_scenario(intent_file_address, input_config_file_address, input_topology_file_address, scenario, updated_spec_run_directory, scoring_keys_address)
  thread_id = message_thread.id
  # run the thread
  run = client.beta.threads.runs.create_and_poll(
    temperature = 0,
    thread_id = thread_id,
    assistant_id = assist_num
  )
  print(f"\n-----------[VISUALIZATION: {visualization}, DIAGRAM TYPE: {diagram_type}, SCENARIO: {scenario}]--------------\n")
  output_file.write(f"\n----------[VISUALIZATION: {visualization}, DIAGRAM TYPE: {diagram_type}, SCENARIO: {scenario}, RUN:{run_number}]-------------------\n")
  if run.status == 'completed':
    messages = client.beta.threads.messages.list(
    thread_id=thread_id
    )
    for message in messages.data:
      if message.role == "assistant":
        messg_id = message.id
        value = message.content[0].text.value
        output_file.write(f"{value}\n")
        print(value)
  output_file.close()

### **NOTICE!** The grades which were received from the output of the evaluation were **manually** extracted and added to the tracking file!

# Tracking Execution

## Topology Understanding Tracking


In [ ]:
from inspect import EndOfBlock
import os.path
import pandas as pd
import time


def topology_understanding_tracking(run_num):
  if not os.path.isfile(f"/content/Run{run_number}_tracking.csv"):
    df = pd.read_csv("/content/Dataset/specs/GeNet_result_template.csv")
  else:
    df = pd.read_csv(f"/content/Run{run_number}_tracking.csv")
  topology_understanding_output_dir = "/content/topology_understanding_output"

  df_dict = df.to_dict('index')
  for index, row in df_dict.items(): # Unpack the tuple into index and row_data
    visualization = row["Visualization"]
    diagram_type = row["Diagram_Type"]
    scenario = row["Scenario"]
    scenario_type = row["Scenario_Type"]
    scenario_dir = os.path.join(topology_understanding_output_dir, scenario_type, visualization, diagram_type, scenario)
    requested_file = f"Topology(Run{run_num}).txt"
    file_path = os.path.join(scenario_dir, requested_file)
    if os.path.isfile(file_path):
      row["Topology_Understanding"] = 'V'
    else:
      row["Topology_Understanding"] = 'X'
  df = pd.DataFrame.from_dict(df_dict, orient='index')
  csv_file_path = f"/content/Run{run_number}_tracking.csv" # Specify the desired file path
  df.to_csv(csv_file_path, index = False)


In [ ]:
topology_understanding_tracking(run_number) # can be changed according to your needs

In [ ]:
def topology_understanding_to_json_tracking(run_num):
  if not os.path.isfile(f"/content/Run{run_number}_tracking.csv"):
    df = pd.read_csv("/content/Dataset/specs/GeNet_result_template.csv")
  else:
    df = pd.read_csv(f"/content/Run{run_number}_tracking.csv")

  topology_understanding_eval_output_dir = "/content/topology_understanding_result_parsed_to_json"

  df_dict = df.to_dict('index')
  for index, row in df_dict.items(): # Unpack the tuple into index and row_data
    visualization = row["Visualization"]
    diagram_type = row["Diagram_Type"]
    scenario = row["Scenario"]
    scenario_type = row["Scenario_Type"]
    scenario_dir = os.path.join(topology_understanding_eval_output_dir, scenario_type, visualization, diagram_type, scenario)
    requested_file = f"Parsed_Topology(Run{run_num}).json"
    file_path = os.path.join(scenario_dir, requested_file)
    if os.path.isfile(file_path):
      row["Topology_Understanding_Eval"] = 'V'
    else:
      row["Topology_Understanding_Eval"] = 'X'
      print(f"{scenario_type} {visualization} {diagram_type} {scenario} Run{run_num} has failed")
  df = pd.DataFrame.from_dict(df_dict, orient='index')
  csv_file_path = f"/content/Run{run_number}_tracking.csv" # Specify the desired file path
  df.to_csv(csv_file_path, index = False)

In [ ]:
topology_understanding_to_json_tracking(run_number)

##Intent Implementation Tracking


Intent Implementation tracking:


In [ ]:
from inspect import EndOfBlock
import os.path
import pandas as pd
import time


intent_implementation_output_dir = "/content/intent_implementation_results"

def intent_implementation_tracking(run_num):
  if not os.path.isfile(f"/content/Run{run_number}_tracking.csv"):
    df = pd.read_csv("/content/Dataset/specs/GeNet_result_template.csv")
  else:
    df = pd.read_csv(f"/content/Run{run_number}_tracking.csv")

  df_dict = df.to_dict('index')
  for index, row in df_dict.items(): # Unpack the tuple into index and row_data
    visualization = row["Visualization"]
    diagram_type = row["Diagram_Type"]
    scenario = row["Scenario"]
    scenario_type = row["Scenario_Type"]
    run_related_dir = os.path.join(intent_implementation_output_dir, scenario_type, visualization, diagram_type, scenario)
    base_directory =os.path.join(run_related_dir, f"Run{run_num}")
    if os.path.isdir(base_directory):
      # List files in the folder
      files = [f for f in os.listdir(base_directory) if os.path.isfile(os.path.join(base_directory, f))]
      # Check if the folder contains exactly three files
      if len(files) >= 1:
        row["Intent_Implementation"] = "V"
      else:
        row["Intent_Implementation"] = "X"
    else:
      row["Intent_Implementation"] = "X"
  df = pd.DataFrame.from_dict(df_dict, orient='index')
  csv_file_path = f"/content/Run{run_number}_tracking.csv" # Specify the desired file path
  df.to_csv(csv_file_path, index = False)

In [ ]:
intent_implementation_tracking(run_number)

Intent Implementation eval tracking:

In [ ]:
import math

def intent_implementation_tracking(run_num):
  if not os.path.isfile(f"/content/Run{run_number}_tracking.csv"):
    df = pd.read_csv("/content/Dataset/specs/GeNet_result_template.csv")
  else:
    df = pd.read_csv(f"/content/Run{run_number}_tracking.csv")

  df_dict = df.to_dict('index')
  for index, row in df_dict.items(): # Unpack the tuple into index and row_data
    if math.isnan(row["Grade"]):
      print(str(index)+ "is nan")
      row["Intent_Implementation_Eval"] = 'X'
    else:
      row["Intent_Implementation_Eval"] = 'V'
  df = pd.DataFrame.from_dict(df_dict, orient='index')
  csv_file_path = f"/content/Run{run_number}_tracking.csv" # Specify the desired file path
  df.to_csv(csv_file_path, index = False)

In [ ]:
intent_implementation_tracking(run_number)